## How BLE works

http://www.blesstags.eu/2018/08/services-characteristics-descriptors.html

A BLE device has services, characteristics, and descriptors in the following tree structure:

```{mermaid}
    graph LR 
    A[BLE Device] --> S1[Service]
    A --> S2[Service]

    S1 --> C1[Characteristic]
    C1 --> D1[Descriptor]
    C1 --> D2[Descriptor]


    S1 --> C2[Characteristic]

    S2 --> C3[Characteristic]
    C3 --> D3[Descriptor]

    S2 --> C4[Characteristic]

```

The behavior of a BLE device is given by the different **Services** that encapsulate different parts of this behavior

Mainly, a service is a container for different data items – these data items are called are called **Characteristics**.

A **Characteristic** is a data item that can be either:
* __read__ (e.g. a sensor value, battery level, etc.) 
* or __written__ (e.g. a configuration value, etc.). 

**Descriptor** - ?

In [ ]:
# | export
import asyncio

import aranet4


async def second():
    await asyncio.sleep(5)
    print("this is another task")


async def hello():
    print("Hello world!")
    await asyncio.sleep(5)
    print("Hello world! x2")
    await second()
    print("Hello again!")

<Task pending name='Task-4' coro=<hello() running at /var/folders/8f/_n2mfgvj0s740c92qff7y2lm0000gp/T/ipykernel_50531/240139731.py:7>>

Hello world!


In [ ]:
asyncio.create_task(hello())

In [ ]:
# | notest
scanned_devices = {}


def on_scan(advertisement):
    if advertisement.device.address not in scanned_devices:
        print(f"Found new device:  {advertisement.device.name}")

    scanned_devices[advertisement.device.address] = advertisement


def print_advertisement(advertisement):
    print("=======================================")
    print(f"  Name:              {advertisement.device.name}")
    print(f"  Address:           {advertisement.device.address}")

    if advertisement.manufacturer_data:
        mf_data = advertisement.manufacturer_data
        print(f"  Version:           {mf_data.version}")
        print(f"  Integrations:      {mf_data.integrations}")
        # print(f"  Disconnected:      {mf_data.disconnected}")
        # print(f"  Calibration state: {mf_data.calibration_state.name}")
        # print(f"  DFU Active:        {mf_data.dfu_active:}")

    print(f"  RSSI:              {advertisement.device.rssi} dBm")

    if advertisement.readings:
        readings = advertisement.readings
        print("-------------------------------------")
        print(f"  CO2:           {readings.co2} pm")
        print(f"  Temperature:   {readings.temperature:.01f} \u00b0C")
        print(f"  Humidity:      {readings.humidity} %")
        print(f"  Pressure:      {readings.pressure:.01f} hPa")
        print(f"  Battery:       {readings.battery} &")
        print(f"  Status disp.:  {readings.status.name}")
        print(f"  Ago:           {readings.ago} s")
    print()


async def run():
    # Scan for 10 seconds, then print results
    print("Scanning Aranet4 devices...")
    aranet4.client.find_nearby(on_scan, 5)
    print(f"\nFound {len(scanned_devices)} devices:\n")

    for addr in scanned_devices:
        advertisement = scanned_devices[addr]
        print_advertisement(advertisement)


asyncio.create_task(run())

<Task pending name='Task-8' coro=<run() running at /var/folders/8f/_n2mfgvj0s740c92qff7y2lm0000gp/T/ipykernel_50531/1269323651.py:39>>

Scanning Aranet4 devices...


In [ ]:
# | notest
import asyncio

from bleak import BleakScanner

aranet_devices = list()


async def find():
    print("start")
    devices = await BleakScanner.discover()
    global dev
    for d in devices:
        if d.name is not None:
            if d.name.lower().startswith("aranet"):
                print("found aranet device", d.name, d.address)
                aranet_devices.append(d)
    print("end")


await find()

In [ ]:
# | notest
from rich import inspect

inspect(aranet_devices[0])
d = aranet_devices[0]
d

In [ ]:
import asyncio

import pandas as pd
# | notest
from bleak import BleakClient

async with BleakClient(
    "2FFB608D-D620-E087-1DF1-434B6F6845B4", max_write_without_response_size=300
) as client:
    # inspect(client)
    # inspect(client.services.services)
    records = list()
    for _, service in client.services.services.items():
        # inspect(service)
        for char in service.characteristics:
            # inspect(char)
            r = dict(
                service=service.description,
                char_descr=char.description,
                char_handle=char.handle,
                properties=char.properties,
            )

            # print('descr', char.description)
            # print('properties', char.properties)
            if "read" in char.properties:
                r["bytes_read"] = bytes(await client.read_gatt_char(char))

            records.append(r)

In [ ]:
df = pd.DataFrame(records)

is_bytes_string = df.char_descr.str.endswith("String")
df.loc[is_bytes_string, "char_value"] = df.loc[is_bytes_string, "bytes_read"].apply(
    lambda x: x.decode("utf-8")
)

is_hex = (df.bytes_read.notna()) & (~is_bytes_string)
df.loc[is_hex, "char_value"] = df.loc[is_hex, "bytes_read"].apply(
    lambda x: int.from_bytes(x, byteorder="little")
)

df

,service,char_descr,char_handle,properties,bytes_read,char_value
0,Nordic Semiconductor ASA,Buttonless DFU,15,"[write, indicate]",NaN,NaN
1,Battery Service,Battery Level,19,"[read, notify]",b'[',91
2,Device Information,Manufacturer Name String,23,[read],b'SAF Tehnika',SAF Tehnika
3,Device Information,Model Number String,25,[read],b'Aranet4',Aranet4
4,Device Information,Serial Number String,27,[read],b'317960115190',317960115190
5,Device Information,Hardware Revision String,29,[read],b'12',12
6,Device Information,Firmware Revision String,31,[read],b'v0.4.14',v0.4.14
7,Device Information,Software Revision String,33,[read],b'v0.4.14',v0.4.14
8,Device Information,System ID,35,[read],b'\xf6S\xe6\x07J\xa6\x04\x00',1308736797168630
9,Unknown,Unknown,38,[read],b'\xf1`\x81\x00\xe8\x03x\x05\x84\x03x\x05',1692512749924216445555794161


In [ ]:
df.loc[18, "char_value"]

60249519959758621862823179573612090156242299038357688108270379730354312780092800047312931963856928514245532617635745839528822256953380003333989247555053966121534709185612642314901342787938857809552212491885933665445494012904190882909259155245974813575240907846028393753023462318231611071953510546275593623438854454524505959692138213830875492882783029887283148220836430060584387709091791820347553156039402745800754865993571917604828641281951176346257965544119318212855702738353737870246652140968592757110032905956401095715432405383414282081989655994454365782890384717055753757950721731587

In [ ]:
# convert pandas column to dummy cols

# pd.get_dummies(df['properties'].explode()).add_prefix('properties_').astype(bool)
df.assign(char_value=df.bytes_read.apply(decode_bytes))

,service,characteristic,properties,bytes_read,char_value
0,Nordic Semiconductor ASA,Buttonless DFU,"[write, indicate]",NaN,NaN
1,Battery Service,Battery Level,"[read, notify]",b'[',NaN
2,Device Information,Manufacturer Name String,[read],b'SAF Tehnika',NaN
3,Device Information,Model Number String,[read],b'Aranet4',NaN
4,Device Information,Serial Number String,[read],b'317960115190',5.439737e+13
5,Device Information,Hardware Revision String,[read],b'12',1.800000e+01
6,Device Information,Firmware Revision String,[read],b'v0.4.14',NaN
7,Device Information,Software Revision String,[read],b'v0.4.14',NaN
8,Device Information,System ID,[read],b'\xf6S\xe6\x07J\xa6\x04\x00',NaN
9,Unknown,Unknown,[read],b'\xf1`\x81\x00\xe8\x03x\x05\x84\x03x\x05',NaN


In [ ]:
import numpy as np

df.loc[0, "bytes_read"] is np.nan

True

In [ ]:
df.bytes_read.apply(decode_bytes).values

array([nan, '[', 'SAF Tehnika', 'Aranet4', 54397372551568, 18, 'v0.4.14',
       'v0.4.14', None, None, nan, None, None, ',\x01',
       '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [ ]:
df.loc[9, "bytes_read"]

# convert hex to int

1692512749924216445555794161